# Testing Model

## Ambil Pipeline model 

In [1]:
import joblib
import pandas as pd

# Load dataset & model
df = pd.read_excel("dataset_labeled_wsn.xlsx")
X = df.drop(columns=["label"])
y = df["label"]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

model_sim = joblib.load("rf_wsn_multiclass_best_pipeline.pkl")


# Real-Time Simulation Test

In [2]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Ambil subset untuk simulasi (opsional, misal 300 baris)
simulasi_X = X_test.sample(300, random_state=42)
simulasi_y = y_test.loc[simulasi_X.index]

# Simulasi "real-time": prediksi baris demi baris
preds = []
for i in range(len(simulasi_X)):
    row = simulasi_X.iloc[i:i+1]     # DataFrame 1 baris
    pred = model_sim.predict(row)[0] # ambil label
    preds.append(pred)

# Evaluasi performa simulasi
print("=== Replay / Real-Time Simulation Test ===")
print("Accuracy:", accuracy_score(simulasi_y, preds))
print("\nClassification Report:")
print(classification_report(simulasi_y, preds))

print("Confusion Matrix:")
print(confusion_matrix(simulasi_y, preds, labels=sorted(y.unique())))


=== Replay / Real-Time Simulation Test ===
Accuracy: 0.9966666666666667

Classification Report:
               precision    recall  f1-score   support

   comm_fault       1.00      1.00      1.00         3
 energy_fault       1.00      1.00      1.00        27
gateway_fault       1.00      0.98      0.99        50
   loop_fault       1.00      1.00      1.00        15
       normal       0.99      1.00      1.00       161
routing_fault       1.00      1.00      1.00        44

     accuracy                           1.00       300
    macro avg       1.00      1.00      1.00       300
 weighted avg       1.00      1.00      1.00       300

Confusion Matrix:
[[  3   0   0   0   0   0]
 [  0  27   0   0   0   0]
 [  0   0  49   0   1   0]
 [  0   0   0  15   0   0]
 [  0   0   0   0 161   0]
 [  0   0   0   0   0  44]]


# Scenario-Based Simulation

In [3]:
import pandas as pd

labels = sorted(y.unique())
samples_per_class = 10   # jumlah contoh per kelas

rows = []

for label in labels:
    # ambil sampel dari test set untuk kelas tertentu
    idx = y_test[y_test == label].sample(samples_per_class, random_state=42).index
    X_subset = X_test.loc[idx]
    y_subset = y_test.loc[idx]

    y_pred = model_sim.predict(X_subset)

    for i, (true_l, pred_l) in enumerate(zip(y_subset, y_pred)):
        rows.append({
            "kelas_sebenarnya": true_l,
            "prediksi_model": pred_l,
            "benar?": (true_l == pred_l)
        })

scenario_df = pd.DataFrame(rows)
print("=== Scenario-Based Simulation (per kelas) ===")
print(scenario_df)

# Rekap berapa yang benar per kelas
print("\nRekap benar per kelas:")
print(scenario_df.groupby("kelas_sebenarnya")["benar?"].mean())


=== Scenario-Based Simulation (per kelas) ===
   kelas_sebenarnya prediksi_model  benar?
0        comm_fault     comm_fault    True
1        comm_fault     comm_fault    True
2        comm_fault     comm_fault    True
3        comm_fault     comm_fault    True
4        comm_fault     comm_fault    True
5        comm_fault     comm_fault    True
6        comm_fault     comm_fault    True
7        comm_fault     comm_fault    True
8        comm_fault     comm_fault    True
9        comm_fault     comm_fault    True
10     energy_fault   energy_fault    True
11     energy_fault   energy_fault    True
12     energy_fault   energy_fault    True
13     energy_fault   energy_fault    True
14     energy_fault   energy_fault    True
15     energy_fault   energy_fault    True
16     energy_fault   energy_fault    True
17     energy_fault   energy_fault    True
18     energy_fault   energy_fault    True
19     energy_fault   energy_fault    True
20    gateway_fault  gateway_fault    True
21    ga

# Stress Test

In [4]:
import numpy as np

# Ambil satu contoh node normal dari test set
normal_idx = y_test[y_test == "normal"].sample(1, random_state=42).index[0]
base_row = X_test.loc[normal_idx].copy()

print("=== Base row (NORMAL) ===")
print(base_row)
print("Prediksi awal:", model_sim.predict(base_row.to_frame().T)[0])

results = []

loss_factors = [1, 2, 3, 4]      # faktor pengali packet_loss
delay_factors = [1, 1.5, 2, 3]   # faktor pengali end_to_end_delay

for lf in loss_factors:
    for dfac in delay_factors:
        test_row = base_row.copy()
        test_row["packet_loss"] = base_row["packet_loss"] * lf
        test_row["end_to_end_delay"] = base_row["end_to_end_delay"] * dfac

        pred = model_sim.predict(test_row.to_frame().T)[0]
        proba = model_sim.predict_proba(test_row.to_frame().T)[0]
        max_proba = np.max(proba)

        results.append({
            "loss_factor": lf,
            "delay_factor": dfac,
            "packet_loss": test_row["packet_loss"],
            "end_to_end_delay": test_row["end_to_end_delay"],
            "prediksi": pred,
            "max_proba": max_proba
        })

stress_df = pd.DataFrame(results)
print("\n=== Hasil Stress Test (berdasarkan 1 sampel normal) ===")
print(stress_df)


=== Base row (NORMAL) ===
energy_level         41.717874
energy_drop_rate      2.612064
latency_ms          236.468754
end_to_end_delay    516.633909
packet_loss          26.983124
throughput_kbps     127.826111
Name: 4865, dtype: float64
Prediksi awal: normal

=== Hasil Stress Test (berdasarkan 1 sampel normal) ===
    loss_factor  delay_factor  packet_loss  end_to_end_delay       prediksi  \
0             1           1.0    26.983124        516.633909         normal   
1             1           1.5    26.983124        774.950864         normal   
2             1           2.0    26.983124       1033.267819  routing_fault   
3             1           3.0    26.983124       1549.901728  routing_fault   
4             2           1.0    53.966248        516.633909         normal   
5             2           1.5    53.966248        774.950864         normal   
6             2           2.0    53.966248       1033.267819         normal   
7             2           3.0    53.966248       1